<a href="https://colab.research.google.com/github/Zizzii0/GEE_notebook_Forestry/blob/main/ICGDM2024%20-%20Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import ee
import geemap.core as geemap
import geopandas
import matplotlib.pyplot as plt
import folium
import numpy as np
import json
import pprint
import builtins
import time
from IPython.display import display, HTML
import pandas as pd
import seaborn as sns
import pandas as pd
import concurrent.futures


In [8]:
# Autentikasi Earth Engine menggunakan akun Google Anda.
ee.Authenticate()
# Inisialisasi Earth Engine.
ee.Initialize(project='ee-ziiizii')
sumedang = ee.FeatureCollection('projects/ee-ziiizii/assets/KAB_SUMEDANG')

In [17]:
# Inisialisasi peta
m = geemap.Map()

# Pusat peta di objek Sumedang dan tambahkan layer batas Sumedang
m.centerObject(sumedang, 10)
m.addLayer(sumedang, {'color': 'red'}, 'Batas Sumedang') # Tambahkan warna agar lebih terlihat

# --- Perhitungan Luas Wilayah ---
# Menghitung luas wilayah dalam meter persegi terlebih dahulu (ini nilai dasar dari Earth Engine)
luas_m2 = sumedang.geometry().area().getInfo()

# Mengonversi luas ke hektar dan kilometer persegi
luas_hektar = luas_m2 / 10_000
luas_km2 = luas_m2 / 1_000_000

# Menampilkan hasil perhitungan luas dalam ha dan km²
print(f"Luas wilayah Sumedang:")
print(f"- {luas_hektar:,.2f} ha")
print(f"- {luas_km2:,.2f} km²")

# Tampilkan peta
m

Luas wilayah Sumedang:
- 157,293.87 ha
- 1,572.94 km²


Map(center=[-6.824740538774393, 107.98126413849603], controls=(WidgetControl(options=['position', 'transparent…

In [18]:
# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

Fungsi ini dirancang untuk menerapkan masking awan dan bayangan awan pada citra Landsat 8 menggunakan band QA_PIXEL.

Band QA_PIXEL pada Landsat 8 (dan 9) memiliki beberapa "bit" yang menyimpan informasi penting mengenai kualitas piksel, termasuk keberadaan awan dan bayangannya.

Berikut adalah bit-bit relevan yang digunakan untuk masking:

Bit 2 (Cirrus): Mengidentifikasi keberadaan awan cirrus. Untuk citra yang bersih, nilai bit ini direkomendasikan adalah "Not Cirrus" (bit 0).

Bit 3 (Cloud Shadow): Mengidentifikasi bayangan awan. Untuk piksel yang bersih, bit ini harus 0.

Bit 4 (Clouds): Mengidentifikasi keberadaan awan padat. Untuk piksel yang bersih, bit ini harus 0.

Bit 5 (Cloud Confidence): Ini adalah bit terpisah yang menunjukkan tingkat kepercayaan awan (0 = Tidak ada awan, 1 = Rendah, 2 = Sedang, 3 = Tinggi). Meskipun disebutkan, bit ini tidak digunakan dalam proses masking yang Anda berikan.

Proses masking bekerja dengan membuat sebuah "mask" yang hanya memilih piksel di mana bit-bit yang relevan (Cirrus, Cloud Shadow, Clouds) disetel ke 0. Ini berarti kita mencari piksel yang tidak memiliki awan, bayangan awan, atau cirrus. Fungsi ini kemudian menerapkan mask tersebut ke citra, sehingga piksel yang mengandung awan atau bayangan awan akan dihilangkan (menjadi transparan atau "no data").

In [21]:
# Perbaikan pada fungsi cloud masking
def mask_clouds(image):
    QA_PIXEL = image.select('QA_PIXEL')

    mask = QA_PIXEL.bitwiseAnd(1 << 2).eq(0).And(  # Bit 2 (Cirrus)
           QA_PIXEL.bitwiseAnd(1 << 3).eq(0)).And(  # Bit 3 (Cloud Shadow)
           QA_PIXEL.bitwiseAnd(1 << 4).eq(0))       # Bit 4 (Clouds)

    return image.updateMask(mask)

In [ ]:
def calculateIndices(image):
    blue = image.select('SR_B2')
    green = image.select('SR_B3')
    red = image.select('SR_B4')
    nir = image.select('SR_B5')
    swir1 = image.select('SR_B6')

    # Advanced Vegetation Index (AVI) =  ((𝐵5 + 1) ∗ (65535 − 𝐵4) ∗ (𝐵5 − 𝐵4)) ^ 1/3
    avi = (nir.add(1).multiply(
           ee.Image.constant(65535).subtract(red)).multiply(
           nir.subtract(red))).pow(1/3).rename('AVI')

    # Bare Soil Index (BSI) = (((B6 + B4) - (B5 + B2)) / ((B6 + B4) + (B5 + B2))) × 100 + 100
    bsi = ((red.add(swir1)).subtract(
            nir.add(blue))).divide(
            (red.add(swir1)).add(nir.add(blue))).multiply(
            100).add(100).rename('BSI')

    # Shadow Index (SI) = [(65535 − B2) × (65535 − B3) × (65535 − B4)] ^ 1/3
    si = ((ee.Image.constant(65535).subtract(blue)).multiply(
            ee.Image.constant(65535).subtract(green)).multiply(
            ee.Image.constant(65535).subtract(red))).pow(1/3).rename('SI')

    # Normalized Difference Vegetation Index (NDVI) =  (B5 - B4) / (B5 + B4)
    ndvi = (nir.subtract(red)
            .divide(nir.add(red))
            .rename('NDVI'))

    return image.addBands([avi, bsi, si, ndvi])

In [ ]:
# Fungsi untuk mendapatkan koleksi citra Landsat
def get_landsat_collection(start_date, end_date, region):
    return ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate(start_date, end_date) \
        .filterBounds(region) \
        .map(applyScaleFactors) \
        .map(mask_clouds)

# Dapatkan dataset untuk tahun 2013 dan 2023
dataset2013 = get_landsat_collection('2013-01-01', '2013-12-31', sumedang)
dataset2023 = get_landsat_collection('2023-01-01', '2023-12-31', sumedang)

# Hitung median untuk citra 2013 dan 2023
median2013 = dataset2013.median().clip(sumedang)
median2023 = dataset2023.median().clip(sumedang)

# Fungsi untuk membuat layer stacking dari band 4, 3, dan 2
def create_stacked_image(median_image):
    return median_image.select(['SR_B4', 'SR_B3', 'SR_B2'])

# Membuat layer stacking untuk tahun 2013 dan 2023
stacked_image_2013 = create_stacked_image(median2013)
stacked_image_2023 = create_stacked_image(median2023)

# Fungsi untuk menghitung total luas area berdasarkan stacked image
def calculate_total_area(stacked_image, region):
    area = stacked_image.multiply(ee.Image.pixelArea())  # Menghitung area dengan ukuran piksel
    total_area = area.reduceRegion(
        reducer=ee.Reducer.sum(),  # Menjumlahkan seluruh area
        geometry=region,  # Wilayah yang dipilih
        scale=30,  # Resolusi citra (misalnya 30 meter untuk Landsat)
        maxPixels=1e9  # Jumlah maksimum piksel untuk menghitung
    )
    return total_area

# Menghitung total luas area untuk tahun 2013 dan 2023
total_area_2013 = calculate_total_area(stacked_image_2013, sumedang)
total_area_2023 = calculate_total_area(stacked_image_2023, sumedang)

# Tampilkan hasil total luas untuk tahun 2013 dan 2023
print('Total Luas Area 2013 : ', total_area_2013.getInfo())
print('Total Luas Area 2023 : ', total_area_2023.getInfo())

# Visualisasi dengan layer stacking
viz_432 = {
    'min': 0,
    'max': 0.3,
    'gamma': 1.4
}

# Tampilkan layer stacking di peta
m.addLayer(stacked_image_2013, viz_432, 'Sumedang 2013 RGB')
m.addLayer(stacked_image_2023, viz_432, 'Sumedang 2023 RGB')
display(m)

# Terapkan fungsi calculateIndices pada hasil median
indices2013 = calculateIndices(median2013)
indices2023 = calculateIndices(median2023)

Total Luas Area 2013 :  {'SR_B2': 45709524.27651353, 'SR_B3': 90639965.8386638, 'SR_B4': 75362529.19692159}
Total Luas Area 2023 :  {'SR_B2': 46410777.283118024, 'SR_B3': 89010532.11884595, 'SR_B4': 73238489.69472001}


Map(bottom=831.0, center=[-6.80989455047656, 107.9805008740824], controls=(ZoomControl(options=['position', 'z…

In [ ]:
# Tanya user apakah ingin melanjutkan dengan ekspor
proceed = input("\nApakah Anda ingin mengekspor setiap saluran citra Sumedang tahun 2013 dan 2023 ke Google Drive? (ya/tidak): ")

if proceed.lower() == 'ya':
    # Konfigurasi ekspor gambar
    export_params = {
        'scale': 30,
        'region': sumedang.geometry(),
        'crs': 'EPSG:4326',
        'maxPixels': 1e9
    }

    # Fungsi untuk memantau status ekspor
    def monitor_task(task, description):
        while task.status()['state'] in ['READY', 'RUNNING']:
            print(f"Status ekspor {description}: {task.status()['state']}")
            time.sleep(10)  # Tunggu 10 detik sebelum cek status lagi

        if task.status()['state'] == 'COMPLETED':
            print(f"Ekspor {description} selesai!")
        else:
            print(f"Ekspor {description} gagal dengan status: {task.status()['state']}")
            if 'error_message' in task.status():
                print(f"Pesan error: {task.status()['error_message']}")

    # Ekstrak nama band dari citra
    band_names = median2013.bandNames().getInfo()

    # Membuat list task ekspor untuk setiap band di tahun 2013 dan 2023
    tasks = []
    for band in band_names:
        # Ekspor untuk tahun 2013
        tasks.append((ee.batch.Export.image.toDrive(
            image=median2013.select(band),
            description=f'Sumedang_2013_{band}',
            folder='ICGDMPart4',
            **export_params
        ), f'Citra Sumedang 2013 Band {band}'))

        # Ekspor untuk tahun 2023
        tasks.append((ee.batch.Export.image.toDrive(
            image=median2023.select(band),
            description=f'Sumedang_2023_{band}',
            folder='ICGDMPart4',
            **export_params
        ), f'Citra Sumedang 2023 Band {band}'))

    # Memulai semua task
    for task, description in tasks:
        task.start()
        print(f"\nMemulai ekspor {description}...")

    # Monitoring semua task
    import concurrent.futures

    print("\nMemulai monitoring proses ekspor...")
    with concurrent.futures.ThreadPoolExecutor(max_workers=len(tasks)) as executor:
        futures = [executor.submit(monitor_task, task, desc) for task, desc in tasks]
        concurrent.futures.wait(futures)

    print("\nSemua proses ekspor telah selesai!")
    print("Silakan cek folder 'ICGDMPart4' di Google Drive Anda.")
else:
    print("\nProses ekspor dibatalkan.")


Apakah Anda ingin mengekspor setiap saluran citra Sumedang tahun 2013 dan 2023 ke Google Drive? (ya/tidak): tidak

Proses ekspor dibatalkan.


In [ ]:
# Tanya user apakah ingin melanjutkan dengan ekspor
proceed = input("\nApakah Anda ingin mengekspor citra Sumedang tahun 2013 dan 2023 ke Google Drive? (ya/tidak): ")

if proceed.lower() == 'ya':
    # Konfigurasi ekspor gambar
    export_params = {
        'scale': 30,
        'region': sumedang.geometry(),
        'crs': 'EPSG:4326',
        'maxPixels': 1e9
    }

    # Fungsi untuk memantau status ekspor
    def monitor_task(task, description):
        while task.status()['state'] in ['READY', 'RUNNING']:
            print(f"Status ekspor {description}: {task.status()['state']}")
            time.sleep(10)  # Tunggu 10 detik sebelum cek status lagi

        if task.status()['state'] == 'COMPLETED':
            print(f"Ekspor {description} selesai!")
        else:
            print(f"Ekspor {description} gagal dengan status: {task.status()['state']}")
            if 'error_message' in task.status():
                print(f"Pesan error: {task.status()['error_message']}")

    # Ekspor gambar dengan monitoring
    tasks = [
        (ee.batch.Export.image.toDrive(
            image=median2013,
            description='Sumedang_2013Part6',
            folder='ICGDMPart6',
            **export_params
        ), 'Citra Sumedang 2013 RGB Part6'),

        (ee.batch.Export.image.toDrive(
            image=median2023,
            description='Sumedang_2023Part6',
            folder='ICGDMPart6',
            **export_params
        ), 'Citra Sumedang 2023 RGB Part6')
    ]

    # Memulai semua task
    for task, description in tasks:
        task.start()
        print(f"\nMemulai ekspor {description}...")

    # Monitoring semua task
    import concurrent.futures

    print("\nMemulai monitoring proses ekspor...")
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        futures = [executor.submit(monitor_task, task, desc) for task, desc in tasks]
        concurrent.futures.wait(futures)

    print("\nSemua proses ekspor telah selesai!")
    print("Silakan cek folder 'ICGDMPart3' di Google Drive Anda.")
else:
    print("\nProses ekspor dibatalkan.")


Apakah Anda ingin mengekspor citra Sumedang tahun 2013 dan 2023 ke Google Drive? (ya/tidak): tidak

Proses ekspor dibatalkan.


In [ ]:
# Ekstrak masing-masing indeks untuk 2013
avi2013 = indices2013.select('AVI')
bsi2013 = indices2013.select('BSI')
si2013 = indices2013.select('SI')
ti2013 = dataset2013.median().clip(sumedang).select('ST_B10')
ndvi2013 = indices2013.select('NDVI')

# Ekstrak masing-masing indeks untuk 2023
avi2023 = indices2023.select('AVI')
bsi2023 = indices2023.select('BSI')
si2023 = indices2023.select('SI')
ti2023 = dataset2023.median().clip(sumedang).select('ST_B10')
ndvi2023 = indices2023.select('NDVI')

In [ ]:
# Inisialisasi peta untuk NDVI
def analyze_ndvi(sumedang):
    m = geemap.Map()
    m.centerObject(sumedang, 11)

    def calculate_min_max(ndvi, region, scale):
        try:
            result = ndvi.reduceRegion(
                reducer=ee.Reducer.minMax(),
                geometry=region,
                scale=scale
            )
            return result.getInfo()
        except Exception as e:
            print(f"Error menghitung min-max: {e}")
            return None

    def calculate_area(ndvi, region):
        try:
            area = ndvi.mask().multiply(ee.Image.pixelArea())
            total_area = area.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=region,
                scale=30,
                maxPixels=1e9
            )
            return total_area
        except Exception as e:
            print(f"Error menghitung area: {e}")
            return None

    # Menghitung nilai min-max untuk NDVI 2013 dan 2023
    min_max_2013 = calculate_min_max(ndvi2013, sumedang, 30)
    min_max_2023 = calculate_min_max(ndvi2023, sumedang, 30)

    # Validasi perhitungan min-max
    if not min_max_2013 or not min_max_2023:
        print("Gagal menghitung rentang nilai NDVI")
        return None

    # Cetak statistik min-max
    print("Rentang nilai NDVI:")
    print(f"NDVI 2013: {min_max_2013}")
    print(f"NDVI 2023: {min_max_2023}")

    # Rentang nilai global untuk visualisasi
    global_min = min(min_max_2013['NDVI_min'], min_max_2023['NDVI_min'])
    global_max = max(min_max_2013['NDVI_max'], min_max_2023['NDVI_max'])

    # Visualisasi NDVI
    viz_ndvi = {
        'min': global_min,
        'max': global_max,
        'palette': ['#d7191c', '#fdae61', '#ffffbf', '#a6d96a', '#1a9641']
    }

    # Tambahkan layer ke peta
    m.addLayer(ndvi2013, viz_ndvi, 'NDVI 2013')
    m.addLayer(ndvi2023, viz_ndvi, 'NDVI 2023')
    display(m)

    # Hitung area untuk kedua tahun (menggunakan NDVI bukan BSI)
    area_2013_ndvi = calculate_area(ndvi2013, sumedang)
    area_2023_ndvi = calculate_area(ndvi2023, sumedang)

    # Validasi perhitungan area
    if not area_2013_ndvi or not area_2023_ndvi:
        print("Gagal menghitung area NDVI")
        return None

    # Konversi ke km²
    area_2013_ndvi_km2 = area_2013_ndvi.get('NDVI').getInfo() / 1e6
    area_2023_ndvi_km2 = area_2023_ndvi.get('NDVI').getInfo() / 1e6

    # Persiapkan data untuk ekspor
    results = {
        'Tahun': ['2013', '2023'],
        'Area_ndvi_km2': [area_2013_ndvi_km2, area_2023_ndvi_km2],
    }

    # Buat DataFrame
    df_results = pd.DataFrame(results)
    print("\nHasil Perhitungan:")
    print(df_results.to_string(index=False))

    return df_results, ndvi2013, ndvi2023

def export_ndvi_data(df_results, ndvi2013, ndvi2023, sumedang):
    # Input validasi
    while True:
        proceed = input("\nApakah Anda ingin melanjutkan dengan ekspor data? (ya/tidak): ").lower()
        if proceed in ['ya', 'tidak']:
            break
        print("Input tidak valid. Silakan masukkan 'ya' atau 'tidak'.")

    if proceed == 'ya':
        # Ekspor ke CSV
        df_results.to_csv('hasil_perhitungan_ndvi_sumedang.csv', index=False)  # Perbaikan nama file

        # Konfigurasi ekspor gambar
        export_params = {
            'scale': 30,
            'region': sumedang.geometry(),
            'crs': 'EPSG:4326',
            'maxPixels': 1e9
        }

        def monitor_task(task, description):
            try:
                while task.status()['state'] in ['READY', 'RUNNING']:
                    print(f"Status ekspor {description}: {task.status()['state']}")
                    time.sleep(10)

                final_status = task.status()['state']
                if final_status == 'COMPLETED':
                    print(f"Ekspor {description} selesai!")
                else:
                    print(f"Ekspor {description} gagal dengan status: {final_status}")
                    if 'error_message' in task.status():
                        print(f"Pesan error: {task.status()['error_message']}")
            except Exception as e:
                print(f"Error monitoring task {description}: {e}")

        # Ekspor gambar dengan monitoring
        tasks = [
            (ee.batch.Export.image.toDrive(
                image=ndvi2013,
                description='NDVI_2013_Part6',
                folder='ICGDMPart6',
                **export_params
            ), 'NDVI 2013 Part6'),
            (ee.batch.Export.image.toDrive(
                image=ndvi2023,
                description='NDVI_2023_Part6',
                folder='ICGDMPart6',
                **export_params
            ), 'NDVI 2023 Part6')
        ]

        # Memulai dan monitoring task
        for task, description in tasks:
            task.start()
            print(f"\nMemulai ekspor {description}...")

        print("\nMemulai monitoring proses ekspor...")
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            futures = [executor.submit(monitor_task, task, desc) for task, desc in tasks]
            concurrent.futures.wait(futures)

        print("\nSemua proses ekspor telah selesai!")
        print("File CSV telah disimpan sebagai 'hasil_perhitungan_ndvi_sumedang.csv'")
        print("Silakan cek folder 'ICGDMPart6' di Google Drive Anda.")
    else:
        print("\nProses ekspor dibatalkan.")

# Jalankan analisis
result = analyze_ndvi(sumedang)
if result:
    df_results, ndvi2013, ndvi2023 = result
    export_ndvi_data(df_results, ndvi2013, ndvi2023, sumedang)

Rentang nilai NDVI:
NDVI 2013: {'NDVI_max': 1.1571931069137773, 'NDVI_min': -0.18207089518341527}
NDVI 2023: {'NDVI_max': 0.9077351156483688, 'NDVI_min': -0.4698567180319004}


Map(center=[-6.824740538774393, 107.98126413849603], controls=(ZoomControl(options=['position', 'zoom_in_text'…


Hasil Perhitungan:
Tahun  Area_ndvi_km2
 2013    1564.441914
 2023    1565.008791

Apakah Anda ingin melanjutkan dengan ekspor data? (ya/tidak): ya

Memulai ekspor NDVI 2013 Part6...

Memulai ekspor NDVI 2023 Part6...

Memulai monitoring proses ekspor...
Status ekspor NDVI 2013 Part6: READY
Status ekspor NDVI 2023 Part6: READY
Status ekspor NDVI 2013 Part6: RUNNING
Status ekspor NDVI 2023 Part6: RUNNING
Status ekspor NDVI 2013 Part6: RUNNING
Status ekspor NDVI 2023 Part6: RUNNING
Status ekspor NDVI 2013 Part6: RUNNING
Status ekspor NDVI 2023 Part6: RUNNING
Status ekspor NDVI 2013 Part6: RUNNING
Status ekspor NDVI 2023 Part6: RUNNING
Status ekspor NDVI 2013 Part6: RUNNING
Status ekspor NDVI 2023 Part6: RUNNING
Status ekspor NDVI 2013 Part6: RUNNING
Status ekspor NDVI 2023 Part6: RUNNING
Status ekspor NDVI 2013 Part6: RUNNING
Status ekspor NDVI 2023 Part6: RUNNING
Status ekspor NDVI 2013 Part6: RUNNING
Status ekspor NDVI 2023 Part6: RUNNING
Status ekspor NDVI 2023 Part6: RUNNING
Status 

In [ ]:
# Inisialisasi peta
m = geemap.Map()
m.centerObject(sumedang, 11)

def calculate_min_max(avi, region, scale):
    try:
        result = avi.reduceRegion(
            reducer=ee.Reducer.minMax(),
            geometry=region,
            scale=scale
        )
        return result.getInfo()
    except Exception as e:
        print(f"Error calculating min-max: {e}")
        return None

def calculate_area(avi, region):
    try:
        area = avi.mask().multiply(ee.Image.pixelArea())
        total_area = area.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=region,
            scale=30,
            maxPixels=1e9
        )
        return total_area
    except Exception as e:
        print(f"Error calculating area: {e}")
        return None

# Menghitung nilai min-max untuk AVI 2013 dan 2023
min_max_2013 = calculate_min_max(avi2013, sumedang, 30)
min_max_2023 = calculate_min_max(avi2023, sumedang, 30)

# Check min-max calculations
if min_max_2013 and min_max_2023:
    global_min = min(min_max_2013['AVI_min'], min_max_2023['AVI_min'])
    global_max = max(min_max_2013['AVI_max'], min_max_2023['AVI_max'])

    # Visualisasi parameters
    vis_params = {
        'min': global_min,
        'max': global_max,
        'palette': ['red', 'yellow', 'green']
    }

    # Tambahkan layer ke peta
    m.addLayer(avi2013, vis_params, 'AVI 2013')
    m.addLayer(avi2023, vis_params, 'AVI 2023')

    # Tampilkan peta
    display(m)
else:
    print("Gagal menghitung nilai min-max")
    exit()

# Hitung area untuk kedua metode
area_2013_area = calculate_area(avi2013, sumedang)
area_2023_area = calculate_area(avi2023, sumedang)

# Check area calculations
if area_2013_area and area_2023_area:
    area_2013_area_km2 = area_2013_area.get('AVI').getInfo() / 1e6
    area_2023_area_km2 = area_2023_area.get('AVI').getInfo() / 1e6

    # Persiapkan data untuk ekspor
    results = {
        'Tahun': ['2013', '2023'],
        'Area_km2': [area_2013_area_km2, area_2023_area_km2],
    }

    # Buat DataFrame
    df_results = pd.DataFrame(results)
    print("\nHasil Perhitungan:")
    print(df_results.to_string(index=False))
else:
    print("Gagal menghitung area")
    exit()

# Input validation
while True:
    proceed = input("\nApakah Anda ingin melanjutkan dengan ekspor data? (ya/tidak): ").lower()
    if proceed in ['ya', 'tidak']:
        break
    print("Input tidak valid. Silakan masukkan 'ya' atau 'tidak'.")

if proceed == 'ya':
    # Ekspor ke CSV
    df_results.to_csv('hasil_perhitungan_avi_sumedang.csv', index=False)

    def monitor_task(task, description):
        try:
            while task.status()['state'] in ['READY', 'RUNNING']:
                print(f"Status ekspor {description}: {task.status()['state']}")
                time.sleep(10)  # Tunggu 10 detik sebelum cek status lagi

            final_status = task.status()['state']
            if final_status == 'COMPLETED':
                print(f"Ekspor {description} selesai!")
            else:
                print(f"Ekspor {description} gagal dengan status: {final_status}")
                if 'error_message' in final_status:
                    print(f"Pesan error: {final_status['error_message']}")
        except Exception as e:
            print(f"Error monitoring task {description}: {e}")

    # Konfigurasi ekspor gambar
    export_params = {
        'scale': 30,
        'region': sumedang.geometry(),
        'crs': 'EPSG:4326',
        'maxPixels': 1e9
    }

    # Ekspor gambar dengan monitoring
    tasks = [
        (ee.batch.Export.image.toDrive(
            image=avi2013,
            description='AVI_2013_Part6',
            folder='ICGDMPart6',
            **export_params
        ), 'AVI 2013 Part6'),

        (ee.batch.Export.image.toDrive(
            image=avi2023,
            description='AVI_2023_Part6',
            folder='ICGDMPart6',
            **export_params
        ), 'AVI 2023 Part6')
    ]

    # Memulai semua task
    for task, description in tasks:
        task.start()
        print(f"\nMemulai ekspor {description}...")

    # Monitoring semua task
    print("\nMemulai monitoring proses ekspor...")
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(monitor_task, task, desc) for task, desc in tasks]
        concurrent.futures.wait(futures)

    print("\nSemua proses ekspor telah selesai!")
    print("File CSV telah disimpan sebagai 'hasil_perhitungan_avi_sumedang.csv'")
    print("Silakan cek folder 'ICGDMPart6")
else:
    print("\nProses ekspor dibatalkan.")

Map(center=[-6.824740538774393, 107.98126413849603], controls=(ZoomControl(options=['position', 'zoom_in_text'…


Hasil Perhitungan:
Tahun    Area_km2
 2013 1564.441914
 2023 1565.008791

Apakah Anda ingin melanjutkan dengan ekspor data? (ya/tidak): tidak

Proses ekspor dibatalkan.


In [ ]:
# Inisialisasi peta
def analyze_bsi(sumedang):
    m = geemap.Map()
    m.centerObject(sumedang, 11)

    def calculate_min_max(bsi, region, scale):
        try:
            result = bsi.reduceRegion(
                reducer=ee.Reducer.minMax(),
                geometry=region,
                scale=scale
            )
            return result.getInfo()
        except Exception as e:
            print(f"Error menghitung min-max: {e}")
            return None

    def calculate_area(bsi, region):
        try:
            area = bsi.mask().multiply(ee.Image.pixelArea())
            total_area = area.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=region,
                scale=30,
                maxPixels=1e9
            )
            return total_area
        except Exception as e:
            print(f"Error menghitung area: {e}")
            return None

    # Menghitung nilai min-max untuk BSI 2013 dan 2023
    min_max_2013 = calculate_min_max(bsi2013, sumedang, 30)
    min_max_2023 = calculate_min_max(bsi2023, sumedang, 30)

    # Validasi perhitungan min-max
    if not min_max_2013 or not min_max_2023:
        print("Gagal menghitung rentang nilai BSI")
        return None

    # Cetak statistik min-max
    print("Rentang nilai BSI:")
    print(f"BSI 2013: {min_max_2013}")
    print(f"BSI 2023: {min_max_2023}")

    # Rentang nilai global untuk visualisasi
    global_min = min(min_max_2013['BSI_min'], min_max_2023['BSI_min'])
    global_max = max(min_max_2013['BSI_max'], min_max_2023['BSI_max'])

    # Visualisasi BSI
    viz_bsi = {
        'min': global_min,
        'max': global_max,
        'palette': ['#fff7ec', '#fee8c8', '#fdd49e', '#fdbb84', '#fc8d59', '#d7301f']
    }

    # Tambahkan layer ke peta
    m.addLayer(bsi2013, viz_bsi, 'BSI 2013')
    m.addLayer(bsi2023, viz_bsi, 'BSI 2023')
    display(m)

    # Hitung area untuk kedua tahun
    area_2013_bsi = calculate_area(bsi2013, sumedang)
    area_2023_bsi = calculate_area(bsi2023, sumedang)

    # Validasi perhitungan area
    if not area_2013_bsi or not area_2023_bsi:
        print("Gagal menghitung area BSI")
        return None

    # Konversi ke km²
    area_2013_bsi_km2 = area_2013_bsi.get('BSI').getInfo() / 1e6
    area_2023_bsi_km2 = area_2023_bsi.get('BSI').getInfo() / 1e6

    # Persiapkan data untuk ekspor
    results = {
        'Tahun': ['2013', '2023'],
        'Area_bsi_km2': [area_2013_bsi_km2, area_2023_bsi_km2],
    }

    # Buat DataFrame
    df_results = pd.DataFrame(results)
    print("\nHasil Perhitungan:")
    print(df_results.to_string(index=False))

    return df_results, bsi2013, bsi2023

def export_bsi_data(df_results, bsi2013, bsi2023, sumedang):
    # Input validasi
    while True:
        proceed = input("\nApakah Anda ingin melanjutkan dengan ekspor data? (ya/tidak): ").lower()
        if proceed in ['ya', 'tidak']:
            break
        print("Input tidak valid. Silakan masukkan 'ya' atau 'tidak'.")

    if proceed == 'ya':
        # Ekspor ke CSV
        df_results.to_csv('hasil_perhitungan_bsi_sumedang.csv', index=False)

        # Konfigurasi ekspor gambar
        export_params = {
            'scale': 30,
            'region': sumedang.geometry(),
            'crs': 'EPSG:4326',
            'maxPixels': 1e9
        }

        def monitor_task(task, description):
            try:
                while task.status()['state'] in ['READY', 'RUNNING']:
                    print(f"Status ekspor {description}: {task.status()['state']}")
                    time.sleep(10)

                final_status = task.status()['state']
                if final_status == 'COMPLETED':
                    print(f"Ekspor {description} selesai!")
                else:
                    print(f"Ekspor {description} gagal dengan status: {final_status}")
                    if 'error_message' in task.status():
                        print(f"Pesan error: {task.status()['error_message']}")
            except Exception as e:
                print(f"Error monitoring task {description}: {e}")

        # Ekspor gambar dengan monitoring
        tasks = [
            (ee.batch.Export.image.toDrive(
                image=bsi2013,
                description='BSI_2013_Part6',
                folder='ICGDMPart6',
                **export_params
            ), 'BSI 2013 Part6'),
            (ee.batch.Export.image.toDrive(
                image=bsi2023,
                description='BSI_2023_Part6',
                folder='ICGDMPart6',
                **export_params
            ), 'BSI 2023 Part6')
        ]

        # Memulai dan monitoring task
        for task, description in tasks:
            task.start()
            print(f"\nMemulai ekspor {description}...")

        print("\nMemulai monitoring proses ekspor...")
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            futures = [executor.submit(monitor_task, task, desc) for task, desc in tasks]
            concurrent.futures.wait(futures)

        print("\nSemua proses ekspor telah selesai!")
        print("File CSV telah disimpan sebagai 'hasil_perhitungan_bsi_sumedang.csv'")
        print("Silakan cek folder 'ICGDMPart6' di Google Drive Anda.")
    else:
        print("\nProses ekspor dibatalkan.")

# Jalankan analisis
result = analyze_bsi(sumedang)
if result:
    df_results, bsi2013, bsi2023 = result
    export_bsi_data(df_results, bsi2013, bsi2023, sumedang)

Rentang nilai BSI:
BSI 2013: {'BSI_max': 132.73878263580053, 'BSI_min': 50.38726740341928}
BSI 2023: {'BSI_max': 135.4126931930286, 'BSI_min': 45.0339037983693}


Map(center=[-6.824740538774393, 107.98126413849603], controls=(ZoomControl(options=['position', 'zoom_in_text'…


Hasil Perhitungan:
Tahun  Area_bsi_km2
 2013   1564.441914
 2023   1565.008791

Apakah Anda ingin melanjutkan dengan ekspor data? (ya/tidak): tidak

Proses ekspor dibatalkan.


In [ ]:
def analyze_si(sumedang):
    # Inisialisasi peta
    m = geemap.Map()
    m.centerObject(sumedang, 11)

    def calculate_min_max(si, region, scale):
        try:
            result = si.reduceRegion(
                reducer=ee.Reducer.minMax(),
                geometry=region,
                scale=scale
            )
            return result.getInfo()
        except Exception as e:
            print(f"Error menghitung min-max: {e}")
            return None

    def calculate_area(si, region):
        try:
            area = si.mask().multiply(ee.Image.pixelArea())
            total_area = area.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=region,
                scale=30,
                maxPixels=1e9
            )
            return total_area
        except Exception as e:
            print(f"Error menghitung area: {e}")
            return None

    # Menghitung nilai min-max untuk SI 2013 dan 2023
    min_max_2013 = calculate_min_max(si2013, sumedang, 30)
    min_max_2023 = calculate_min_max(si2023, sumedang, 30)

    # Validasi perhitungan min-max
    if not min_max_2013 or not min_max_2023:
        print("Gagal menghitung rentang nilai SI")
        return None

    # Cetak statistik min-max
    print("Rentang nilai SI:")
    print(f"SI 2013: {min_max_2013}")
    print(f"SI 2023: {min_max_2023}")

    # Rentang nilai global untuk visualisasi
    global_min = min(min_max_2013['SI_min'], min_max_2023['SI_min'])
    global_max = max(min_max_2013['SI_max'], min_max_2023['SI_max'])

    # Visualisasi SI
    viz_si = {
        'min': global_min,
        'max': global_max,
        'palette': ['#ffffff', '#000000']  # Hitam untuk nilai terendah, putih untuk nilai tertinggi
    }

    # Tambahkan layer ke peta
    m.addLayer(si2013, viz_si, 'SI 2013')
    m.addLayer(si2023, viz_si, 'SI 2023')
    display(m)

    # Hitung area untuk kedua tahun
    area_2013_si = calculate_area(si2013, sumedang)
    area_2023_si = calculate_area(si2023, sumedang)

    # Validasi perhitungan area
    if not area_2013_si or not area_2023_si:
        print("Gagal menghitung area SI")
        return None

    # Konversi ke km²
    # KOREKSI: Gunakan variabel SI, bukan BSI seperti pada kode sebelumnya
    area_2013_si_km2 = area_2013_si.get('SI').getInfo() / 1e6
    area_2023_si_km2 = area_2023_si.get('SI').getInfo() / 1e6

    # Persiapkan data untuk ekspor
    results = {
        'Tahun': ['2013', '2023'],
        'Area_tanpa_Threshold_km2': [area_2013_si_km2, area_2023_si_km2]
    }

    # Buat DataFrame
    df_results = pd.DataFrame(results)
    print("\nHasil Perhitungan:")
    print(df_results.to_string(index=False))

    return df_results, si2013, si2023

def export_si_data(df_results, si2013, si2023, sumedang):
    # Input validasi
    while True:
        proceed = input("\nApakah Anda ingin melanjutkan dengan ekspor data? (ya/tidak): ").lower()
        if proceed in ['ya', 'tidak']:
            break
        print("Input tidak valid. Silakan masukkan 'ya' atau 'tidak'.")

    if proceed == 'ya':
        # Ekspor ke CSV
        df_results.to_csv('hasil_perhitungan_si_sumedang.csv', index=False)

        # Konfigurasi ekspor gambar
        export_params = {
            'scale': 30,
            'region': sumedang.geometry(),
            'crs': 'EPSG:4326',
            'maxPixels': 1e9
        }

        def monitor_task(task, description):
            try:
                while task.status()['state'] in ['READY', 'RUNNING']:
                    print(f"Status ekspor {description}: {task.status()['state']}")
                    time.sleep(10)

                final_status = task.status()['state']
                if final_status == 'COMPLETED':
                    print(f"Ekspor {description} selesai!")
                else:
                    print(f"Ekspor {description} gagal dengan status: {final_status}")
                    if 'error_message' in task.status():
                        print(f"Pesan error: {task.status()['error_message']}")
            except Exception as e:
                print(f"Error monitoring task {description}: {e}")

        # Ekspor gambar dengan monitoring
        tasks = [
            (ee.batch.Export.image.toDrive(
                image=si2013,
                description='SI_2013_Part6',
                folder='ICGDMPart6',
                **export_params
            ), 'SI 2013 Part6'),
            (ee.batch.Export.image.toDrive(
                image=si2023,
                description='SI_2023_Part6',
                folder='ICGDMPart6',
                **export_params
            ), 'SI 2023 Part6')
        ]

        # Memulai dan monitoring task
        for task, description in tasks:
            task.start()
            print(f"\nMemulai ekspor {description}...")

        print("\nMemulai monitoring proses ekspor...")
        with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
            futures = [executor.submit(monitor_task, task, desc) for task, desc in tasks]
            concurrent.futures.wait(futures)

        print("\nSemua proses ekspor telah selesai!")
        print("File CSV telah disimpan sebagai 'hasil_perhitungan_si_sumedang.csv'")
        print("Silakan cek folder 'ICGDMPart5' di Google Drive Anda.")
    else:
        print("\nProses ekspor dibatalkan.")

# Jalankan analisis
result = analyze_si(sumedang)
if result:
    df_results, si2013, si2023 = result
    export_si_data(df_results, si2013, si2023, sumedang)

Rentang nilai SI:
SI 2013: {'SI_max': 65535.03594414541, 'SI_min': 65534.460034535354}
SI 2023: {'SI_max': 65534.99348874992, 'SI_min': 65534.50640414001}


Map(center=[-6.824740538774393, 107.98126413849603], controls=(ZoomControl(options=['position', 'zoom_in_text'…


Hasil Perhitungan:
Tahun  Area_tanpa_Threshold_km2
 2013               1564.441914
 2023               1565.008791

Apakah Anda ingin melanjutkan dengan ekspor data? (ya/tidak): tidak

Proses ekspor dibatalkan.


In [ ]:
# Inisialisasi peta
m = geemap.Map()
m.centerObject(sumedang, 11)

# Fungsi untuk menghitung nilai minimum dan maksimum untuk TI
def calculate_min_max(ti, region, scale):
    try:
        result = ti.reduceRegion(
            reducer=ee.Reducer.minMax(),
            geometry=region,
            scale=scale
        )
        return result.getInfo()
    except Exception as e:
        print(f"Error menghitung min-max: {e}")
        return None

# Fungsi untuk menghitung luas area berdasarkan nilai TI
def calculate_area_based_on_ti(ti, region):
    try:
        area = ti.mask().multiply(ee.Image.pixelArea())
        total_area = area.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=region,
            scale=100,
            maxPixels=1e9
        )
        return total_area
    except Exception as e:
        print(f"Error menghitung area: {e}")
        return None

# Menghitung nilai min-max untuk TI 2013 dan 2023
min_max_2013 = calculate_min_max(ti2013, sumedang, 100)
min_max_2023 = calculate_min_max(ti2023, sumedang, 100)

# Validasi hasil min-max
if not min_max_2013 or not min_max_2023:
    print("Gagal menghitung rentang nilai TI")
    exit()

# Cetak statistik min-max
print("Rentang nilai Thermal Index:")
print(f"TI 2013: {min_max_2013}")
print(f"TI 2023: {min_max_2023}")

# Rentang nilai global untuk visualisasi
global_min = min(min_max_2013['ST_B10_min'], min_max_2023['ST_B10_min'])
global_max = max(min_max_2013['ST_B10_max'], min_max_2023['ST_B10_max'])

# Visualisasi TI
viz_ti = {
    'min': global_min,
    'max': global_max,
    'palette': ['#313695', '#4575b4', '#74add1', '#abd9e9', '#fdae61', '#f46d43', '#d73027']  # Gradien warna
}

# Tambahkan layer ke peta
m.addLayer(ti2013, viz_ti, 'TI 2013')
m.addLayer(ti2023, viz_ti, 'TI 2023')
display(m)

# Hitung area untuk kedua tahun
area_2013_ti = calculate_area_based_on_ti(ti2013, sumedang)
area_2023_ti = calculate_area_based_on_ti(ti2023, sumedang)

# Validasi hasil area
if not area_2013_ti or not area_2023_ti:
    print("Gagal menghitung area TI")
    exit()

# Konversi ke km²
area_2013_ti_km2 = area_2013_ti.get('ST_B10').getInfo() / 1e6
area_2023_ti_km2 = area_2023_ti.get('ST_B10').getInfo() / 1e6

# Persiapkan data untuk ekspor
results = {
    'Tahun': ['2013', '2023'],
    'Area_tanpa_Threshold_km2': [area_2013_ti_km2, area_2023_ti_km2]
}

# Buat DataFrame
df_results = pd.DataFrame(results)
print("\nHasil Perhitungan:")
print(df_results.to_string(index=False))

# Tanya user apakah ingin melanjutkan dengan ekspor
proceed = input("\nApakah Anda ingin melanjutkan dengan ekspor data? (ya/tidak): ")

if proceed.lower() == 'ya':
    # Ekspor ke CSV
    df_results.to_csv('hasil_perhitungan_ti_sumedang.csv', index=False)

    # Konfigurasi ekspor gambar
    export_params = {
        'scale': 100,
        'region': sumedang.geometry(),
        'crs': 'EPSG:4326',
        'maxPixels': 1e9
    }

    # Fungsi untuk memantau status ekspor
    def monitor_task(task, description):
        while task.status()['state'] in ['READY', 'RUNNING']:
            print(f"Status ekspor {description}: {task.status()['state']}")
            time.sleep(10)  # Tunggu 10 detik sebelum cek status lagi

        if task.status()['state'] == 'COMPLETED':
            print(f"Ekspor {description} selesai!")
        else:
            print(f"Ekspor {description} gagal dengan status: {task.status()['state']}")
            if 'error_message' in task.status():
                print(f"Pesan error: {task.status()['error_message']}")

    # Ekspor gambar dengan monitoring
    tasks = [
        (ee.batch.Export.image.toDrive(
            image=ti2013,
            description='TI_2013_Part6',
            folder='ICGDMPart6',
            **export_params
        ), 'TI 2013 Part6'),

        (ee.batch.Export.image.toDrive(
            image=ti2023,
            description='TI_2023_Part6',
            folder='ICGDMPart6',
            **export_params
        ), 'TI 2023 Part6')
    ]

    # Memulai semua task
    for task, description in tasks:
        task.start()
        print(f"\nMemulai ekspor {description}...")

    # Monitoring semua task
    print("\nMemulai monitoring proses ekspor...")
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(monitor_task, task, desc) for task, desc in tasks]
        concurrent.futures.wait(futures)

    print("\nSemua proses ekspor telah selesai!")
    print("File CSV telah disimpan sebagai 'hasil_perhitungan_ti_sumedang.csv'")
    print("Silakan cek folder 'ICGDMPart5' di Google Drive Anda.")
else:
    print("\nProses ekspor dibatalkan.")


Rentang nilai Thermal Index:
TI 2013: {'ST_B10_max': 327.32493944, 'ST_B10_min': 286.41124004}
TI 2023: {'ST_B10_max': 324.52558106000004, 'ST_B10_min': 293.28487826}


Map(center=[-6.824740538774393, 107.98126413849603], controls=(ZoomControl(options=['position', 'zoom_in_text'…


Hasil Perhitungan:
Tahun  Area_tanpa_Threshold_km2
 2013               1561.803466
 2023               1562.356558

Apakah Anda ingin melanjutkan dengan ekspor data? (ya/tidak): tidak

Proses ekspor dibatalkan.
